In [17]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [4]:
df_spamCollection = pd.read_csv(r"C:\Users\hongw\Downloads\1574413540_lesson91\SpamCollection", sep='\t', names=['response','message'])

In [5]:
df_spamCollection.head(5)

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df_spamCollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [11]:
df_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [12]:
df_spamCollection['length'] = df_spamCollection['message'].apply(len)

In [13]:
df_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [18]:
def message_text_process(mess):
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hongw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
df_spamCollection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
bag_of_words_transformer = CountVectorizer ( analyzer= message_text_process).fit(df_spamCollection['message'])
bag_of_words_transformer

CountVectorizer(analyzer=<function message_text_process at 0x0000028CB8A1C940>)

In [26]:
print(len(bag_of_words_transformer.vocabulary_))

11425


In [28]:
message_bagofwords = bag_of_words_transformer.transform (df_spamCollection['message'] )
message_bagofwords

<5572x11425 sparse matrix of type '<class 'numpy.int64'>'
	with 50548 stored elements in Compressed Sparse Row format>

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords )

In [31]:
message_tfidf = tfidf_transformer.transform(message_bagofwords)
message_tfidf.shape

(5572, 11425)

In [35]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit (message_tfidf, df_spamCollection['response'])

In [37]:
message = df_spamCollection['message'][4]
bag_of_words_for_message = bag_of_words_transformer.transform ([message])
tfidf = tfidf_transformer.transform (bag_of_words_for_message)

print('predicted',spam_detect_model.predict(tfidf)[0])
print('expected', df_spamCollection.response[4])

predicted ham
expected ham
